## Baseline Model

베이스 라인을 만들 차례입니다. 아래는 베이스 라인 모델을 만드는 과정입니다.

1. Fix the seed value and set up your GPU equipment (시드값 고정 및 GPU 장비 설정)
   - [1] Fix the seed value (시드값 고정)
   - [2] Set up GPU equipment (GPU 장비 설정)
2. Prepare Data (데이터 준비)
   - [1] Separate training/validation data (훈련/검증 데이터 분리)
   - [2] Define dataset classes (데이터셋 클래스 정의)
   - [3] Define image transformer (이미지 변환기 정의)
   - [4] Create datasets & a data loader (데이터셋 및 데이터 로더 생성)
3. Create Model (모델 생성)
   - [1] Pre-trained models and transfer learing (사전 훈련 모델과 전이 학습)
   - [2] Create an EfficientNet model (EfficientNet 모델 생성)
4. Train Model & Validate Performance (모델 훈련 및 성능 검증)
   - [1] Set Loss Function & Optimizer (손실 함수와 옵티마이저 설정)
   - [2] Train & Validate Performance (훈련 및 성능 검증)
5. Predict & Submission (예측 및 제출)
   - [1] Predict (예측)
   - [2] Submission (제출)

## 1. Fix the seed value and set up your GPU equipment (시드값 고정 및 GPU 장비 설정)

### [1] Fix the seed value (시드값 고정)

먼저 시드값을 고정합니다.

In [ ]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

### [2] Set up GPU equipment (GPU 장비 설정)

다음으로 장비를 할당할 차례입니다. 오른쪽 [Settings] 탭에서 Accelerator를 "GPU T4 x2"로 바꾸고, 이어서 다음 코드를 실행합니다.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

## 2. Prepare Data (데이터 준비)

데이터를 먼저 불러옵니다.

In [ ]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### [1] Separate training/validation data (훈련/검증 데이터 분리)

전체 훈련 데이터인 train을 훈련 데이터와 검증 데이터로 분리하겠습니다.

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train,
                               test_size = 0.1,
                               stratify = train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                               random_state = 50)

타깃값이 고루 분포되도록 분리하기 위해 stratify 파라미터에 타깃값 4개를 전달했습니다.

### [2] Define dataset classes (데이터셋 클래스 정의)

데이터셋 클래스를 정의합니다.

In [ ]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir = './', transform = None, is_test = False):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        # (1)
        self.is_test = is_test
    
    # 데이터셋 크기 반환 메서드
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id + '.jpg' # (2) 이미지 파일 경로
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image = image)['image']
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃값도 반환
        if self.is_test: # (4)
            return image # (5) 테스트용일 때
        else:
            # (6) 타깃값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label # (7) 훈련/검증용일 때

(1) 데이터셋을 테스트용으로 만들려면 초기화 메서드의 is_test 파라미터에 True를, 훈련이나 검증용으로 만들려면 False를 전달합니다.

__getitem__() 메서드로 데이터를 가져올 때는 생성 시 지정한 (4) is_test 값을 확인해서, (5) 테스트 데이터용이라면 테스트 데이터에는 타깃값이 없기 때문에 이미지 데이터만 반환하고 (7) 훈련 혹은 검증용이라면 타깃값도 함께 반환합니다.

(6) 훈련 혹은 검증용일 경우 타깃값은 4가지(self.df.iloc[idx, 1:5]) 중 가장 큰 값의 인덱스(np.argmax(...))가 됩니다. 즉, 가장 큰 타깃값이 healthy면 0, multiple_disease면 1, rust면 2, scab이면 3을 label에 할당합니다.

(2)에서는 이미지 파일 경로 끝에 파일 확장자(.jpg)를 덧붙였습니다.

### [3] Define image transformer (이미지 변환기 정의)

데이터 증강용 이미지 변환기를 정의할 차례입니다. 이번에는 albumentations가 제공하는 이미지 변환기를 사용할 겁니다. torchvision의 변환기와 비교했을 때, 처리 속도가 빠르고, 더 다양한 이미지 변환을 제공한다는 장점이 있습니다.

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

'훈련 데이터용' 변환기부터 정의한 후, 바로 이어서 '검증 및 테스트 데이터용'을 정의하겠습니다.

In [ ]:
transform_train = A.Compose([
    A.Resize(450, 650), # (1) 이미지 크기 조절
    A.RandomBrightnessContrast(brightness_limit = 0.2, # (2) 밝기 대비 조절
                               contrast_limit = 0.2, p = 0.3),
    A.VerticalFlip(p = 0.2),   # 상하 대칭 변환
    A.HorizontalFlip(p = 0.5), # 좌우 대칭 변환
    A.ShiftScaleRotate(        # (3) 이동, 스케일링, 회전 변환
        shift_limit = 0.1,
        scale_limit = 0.2,
        rotate_limit = 30, p = 0.3),
    A.OneOf([A.Emboss(p = 1), # (4) 양각화, 날카로움, 블러 효과
             A.Sharpen(p = 1),
             A.Blur(p = 1)], p = 0.3),
    A.PiecewiseAffine(p = 0.3), # (5) 어파인 변환
    A.Normalize(), # (6) 정규화 변환
    ToTensorV2()   # (7) 텐서로 변환
])

(1) Resize : 이미지 크기를 조절하는 변환기

(2) RandomBrightnessContrast : 이미지의 밝기와 대비를 조절하는 변환기

(3) ShiftScaleRotate : 이동, 스케일링, 회전 변환기

(4) 양각화 효과(Emboss), 날카롭게 만드는 효과(Sharpen), 블러 효과(Blur) 중 하나를 선택(OneOf)해 적용합니다.

(5) PiecewiseAffine : 어파인 변환기입니다. 어파인 변환이란 이동, 확대/축소, 회전 등으로 이미지 모양을 전체적으로 바꾸는 변환입니다.

(6) Normalize : 값을 정규화하는 변환기로, torchvision의 transform.Normalize()와 비슷합니다.

(7) ToTesnorV2() : 이미지 데이터를 텐서 형식으로 변환합니다. torchvision의 transform.ToTensor()와 비슷하다고 보면 됩니다.

이어서 '검증 및 테스트 데이터용' 변환기입니다. 필수적인 변환기만 적용해 정의합니다.

In [ ]:
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()
])

크기는 당연히 훈련 데이터와 똑같에 맞추는 게 좋고, 픽셀 값 범위도 비슷해야(정규화해야) 서로 비교하기 쉽습니다. 마지막으로 파이토치는 텐서 객체만 취급하기 때문에 ToTensorV2() 변환기도 꼭 필요합니다.

### [4] Create datasets & a data loader (데이터셋 및 데이터 로더 생성)

데이터 준비의 마지막 단계입니다. 데이터셋부터 정의해봅시다.

In [ ]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir = img_dir, transform = transform_train)
dataset_valid = ImageDataset(valid, img_dir = img_dir, transform = transform_test)

훈련 데이터셋(dataset_train)을 만들 때는 훈련용 변환기(transform_train)를, 검증 데이터셋(dataset_valid)을 만들 때는 검증/테스트용 변환기(transform_test)를 전달했습니다.

이번는 멀티프로세싱을 활용해보겠습니다. 모델 훈련 시간이 꽤 걸리기 때문입니다. 멀티프로세싱을 사용하려면 다음과 같이 데이터 로더의 시드값을 고정해야 합니다. 먼저 seed_worker()를 정의하고 제너레이터를 생성합니다.

In [ ]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

이어서 데이터 로더도 생성합니다. 훈련 데이터가 1,821개로 그렇게 많지 않아서 배치 크게는 4로 작게 설정했습니다.

In [ ]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size = batch_size,
                         shuffle = True, worker_init_fn = seed_worker,
                         generator = g, num_workers = 2)
loader_valid = DataLoader(dataset_valid, batch_size = batch_size,
                         shuffle = False, worker_init_fn = seed_worker,
                         generator = g, num_workers = 2)

## 3. Create Model (모델 생성)

이번에는 모델을 직접 설계하지 않고 사전 훈련된 모델을 전이 학습시키는 방식으로 만들어보겠습니다.

### [1] Pre-trained models and transfer learing (사전 훈련 모델과 전이 학습)

사전 훈련 모델(pretrained model)이란 말 그대로 이미 한 분야에서 훈련을 마친 모델을 일컬으며, 전이 학습(transfer learing)이란 사전 훈련 모델을 유사한 다른 영역에서 재훈련시키는 기법입니다. 비유하자면, 분야 전문가(사전 훈련 모델)를 모셔와서 우리 회사만의 특수한 상황을 알려드린 후(전이 학습) 컨설팅받는 것과 비슷합니다.

### [2] Create an EfficientNet model (EfficientNet 모델 생성)

EfficientNet은 2019년 5월에 개발된 CNN 모델로, 우수한 성능을 보여 주목을 받았습니다. 토론 내용에 따르면 본 경진대회에서는 EfficientNet이 우수한 성능을 보인다는 의견이 많았습니다. 'best single model'을 주제로 토론이 자주 벌어지니 미리 찾아보세요.

Luke Melas-Kyriazi 라는 사람이 EfficientNet을 모듈로 구현해놨습니다. efficient_pytorch 모듈인데, 캐글 환경에서 설치되어 있지 않아서 별도로 설치해야 합니다. 다음 코드로 설치를 진행합니다.

In [ ]:
!pip install efficientnet-pytorch==0.7.1

efficientnet_pytorch 모듈의 EfficientNet 모델을 임포트하고 사전 훈련된 efficientnet-b7을 불러와서 device 장비에 할당합니다.

In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes = 4) # (1)

model = model.to(device) # 장비_할당

(1)에서 efficient-b7을 불러올 때 전달한 num_classes 파라미터는 최종 출력값 개수를 뜻합니다. EfficientNet은 타깃값이 1,000개인 이미지 데이터로 사전 훈련한 모델이므로 num_classes에 아무 값도 전달하지 않으면 최종 출력값이 1,000개가 됩니다. 하지만 본 경진대회에서 예측해야 하는 타깃값은 총 4개이므로 4를 전달했습니다.

## 4. Train Model & Validate Performance (모델 훈련 및 성능 검증)

손실 함수와 옵티마이저를 설정해 훈련시킨 후 성능을 확인해보겠습니다.

### [1] Set Loss Function & Optimizer (손실 함수와 옵티마이저 설정)

손실 함수부터 CrossEntropyLoss()로 정의하겠습니다.

In [ ]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

다음으로 옵티마이저를 정의합니다. 이번에는 AdamW 라는 옵티마이저를 사용해보겠습니다. AdamW는 Adam에 가중치 감쇠를 추가로 적용해서 일반화 성능이 더 우수합니다. 가중치 감쇠(weight decay)란 가중치를 작게 조절하는 규제 기법으로, 과대적합을 억제해줍니다.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 0.00006, weight_decay = 0.0001)

학습률은 0.00006으로 설정했습니다. 여러 학습률을 적용해 실험해보시기를 권장합니다. weight_decay는 가중치 감쇠를 의미하는 파라미터입니다. 여기서는 0.0001로 작은 값을 전달해서 미세하게 규제를 적용했습니다.

### [2] Train & Validate Performance (훈련 및 성능 검증)

'매 에폭마다 검증'하는 방식으로 훈련을 진행하겠습니다. 더 오래 걸리지만, 과대적합 없이 훈련이 잘 되고 있는지 확인할 수 있다는 장점이 있습니다.

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대
epochs = 5

# 총 에폭만큼 반복
for epoch in range(epochs):
    # == [ 훈련 ] ==============================
    model.train()         # 모델을 훈련 상태로 설정
    epoch_train_loss = 0  # 에폭별 손실값 초기화 (훈련 데이터용)
    
    # '반복 횟수'만큼 반복
    for images, labels in tqdm(loader_train): # (2)
        # 이미지, 레이블(타깃값) 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 수차 (훈련 데이터용)
        epoch_train_loss += loss.item()
        loss.backward()  # 역전파 수행
        optimizer.step() # 가중치 갱신
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    # == [ 검증 ] ==============================
    model.eval()          # 모델을 평가 상태로 설정
    epoch_valid_loss = 0  # 에폭별 손실값 초기화 (검증 데이터용)
    preds_list = []       # 예측 확률값 저장용 리스트 초기화
    true_onehot_list = [] # 실제 타깃값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            # 예측 확률값
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            # (5) 실젯값 (원-핫 인코딩 형식)
            true_onehot = torch.eye(4, device = device)[labels].cpu().numpy()
            # (6) 예측 확률값과 실젯값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    # 검증 데이터 손실값 및 ROC AUC 점수 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')

## 5. Predict & Submission (예측 및 제출)

훈련이 끝났으니, 이제 예측을 해야겠죠? 테스트용 데이터셋과 데이터 로더를 생성합니다.

In [ ]:
dataset_test = ImageDataset(test, img_dir = img_dir,
                           transform = transform_test, is_test = True)

loader_test = DataLoader(dataset_test, batch_size = batch_size,
                         shuffle = False, worker_init_fn = seed_worker,
                        generator = g, num_workers = 2)

### [1] Predict (예측)

테스트 데이터를 활용해 타깃 확률을 예측해봅니다.

In [ ]:
model.eval() # 모델을 평가 상태로 설정

preds = np.zeros((len(test), 4)) # (1) 예측값 저장용 배열 초기화

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률
        preds_part = torch.softmax(outputs.cpu(), dim = 1).squeeze().numpy() # (2)
        preds[i * batch_size:(i + 1) * batch_size] += preds_part # (3)

(1) 먼저 예측값을 저장하기 위해 배열을 준비합니다. np.zeros()는 전달받은 형상 크기에 맞게 0으로 채워진 배열을 반환합니다. 인수로 전달한 len(test)와 4는 각각 행과 열의 개수입니다. 타깃값이 4개라서 열이 4개입니다.

(2)와 (3)에서는 타깃 예측 확률을 구합니다. (2) outputs에는 신경망 출력값이 배치 크기만큼 존재합니다. 그 출력값에 소프트맥스 함수를 취해 확률값을 구해 preds_part에 할당했습니다. (3)이 preds_part를 이용해 preds 배열을 갱신합니다. preds 배열에서 해당하는 행 위치에 있는 0을 배치 크기만큼 확률값들로 갱신한 것입니다.

for문이 끝나면 다음 그림과 같이 preds에 모든 테스트 데이터의 예측 확률값들이 저장돼 있을 것입니다.

### [2] Submission (제출)

마지막으로 제출 파일을 만들고 제출해보겠습니다.

In [ ]:
submission[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds
submission.to_csv('submission.csv', index = False)

The End.